In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('..')  # enable import from src/

In [16]:
import re
import pickle
from pathlib import Path

import gurobipy
from gurobipy import GRB
import dgl
import numpy as np
import torch

from src.problem import get_model, get_benders_cut, load_instance, get_feasible

In [6]:
instances = sorted(list(Path('../data/raw').glob('97_9*.jl')))

i = 1
print(instances[i])
instance = load_instance(instances[i])

J = instance['jobs'][0]
T = instance['tamanho'][0]
uso_p = instance['uso_p']
recurso_p = instance['recurso_p']

jobs = list(range(J))
model = get_model(jobs, instance, coupling=True)

with open('../97_9_opts.pkl', 'rb') as f:
    opts = pickle.load(f)

x_opt = opts[instances[i].name]['sol']
x_opt.shape

../data/raw/97_9_1.jl


(1746,)

In [7]:
model_vars = np.core.defchararray.array([v.getAttr(GRB.Attr.VarName) for v in model.getVars()])
model_vars = model_vars[(model_vars.find('x') >= 0) | (model_vars.find('phi') >= 0)]  # drop soc vars
sol = x_opt[model_vars.find('x') == -1]
sol_vars = model_vars[model_vars.find('phi') == -1]

sol_idx = [re.fullmatch(r"x\((\d+),(\d+)\)", s_v).groups() for s_v in sol_vars]
sol_idx = np.array(list(map(lambda jt: list(map(int, jt)), sol_idx)))

In [10]:
x_cand_opt = np.zeros_like(sol).reshape((J, T))
for sol_jt, (j, t) in zip(sol, sol_idx):
    x_cand_opt[j,t] = sol_jt

assert get_benders_cut(instance, x_cand_opt) is None, "Optimal is not BSP feasible!"

Set parameter InfUnbdInfo to value 1


In [11]:
x_cand_dummy = np.ones_like(x_cand_opt)

cut_w, cut_b = get_benders_cut(instance, x_cand_dummy)

print((cut_w * x_cand_dummy).sum() + cut_b >= 0)
print((cut_w * x_cand_opt).sum() + cut_b >= 0)

Set parameter InfUnbdInfo to value 1
False
True


Applying Benders by solving the master with Gurobi:

In [12]:
instances = sorted(list(Path('../data/raw').glob('97_9*.jl')))

i = 0
print(instances[i])
instance = load_instance(instances[i])

J = instance['jobs'][0]
T = instance['tamanho'][0]
uso_p = instance['uso_p']
recurso_p = instance['recurso_p']

with open('../97_9_opts.pkl', 'rb') as f:
    opts = pickle.load(f)

x_opt = opts[instances[i].name]['sol']

# solve problem without coupling constraints
model = get_model(jobs, instance, coupling=False, new_ineq=True)
model.setParam('TimeLimit', 960)
model.optimize()
model.status

../data/raw/97_9.jl


2

In [13]:
x_cand = np.zeros((J,T))
for var in model.getVars():
    var_name = var.getAttr(GRB.Attr.VarName)
    try:
        j, t = re.match(r"x\((\d+),(\d+)\)",var_name).groups()
        j = int(j)
        t = int(t)
        x_cand[j,t] = var.X
    except AttributeError:
        continue
cut_w, cut_b = get_benders_cut(instance, x_cand)

print((cut_w * x_cand).sum() + cut_b >= 0)
print((cut_w * x_cand_opt).sum() + cut_b >= 0)

Set parameter InfUnbdInfo to value 1
False
True


In [14]:
model.addConstr(
    sum(cut_w[j,t] * model.getVarByName(f"x({j},{t})")
        for j in range(J) for t in range(T))
    + cut_b >= 0
)
model.update()
# model.setParam('TimeLimit', 960)
model.optimize()
model.status

2

In [15]:
new_x_cand = np.zeros((J,T))
for var in model.getVars():
    var_name = var.getAttr(GRB.Attr.VarName)
    try:
        j, t = re.match(r"x\((\d+),(\d+)\)",var_name).groups()
        j = int(j)
        t = int(t)
        new_x_cand[j,t] = var.X
    except AttributeError:
        continue
get_benders_cut(instance, new_x_cand)

Set parameter InfUnbdInfo to value 1


Benders with feasibility recovery:

In [36]:
instances = sorted(list(Path('../data/raw').glob('97_9*.jl')))

i = 0
print(instances[i])
instance = load_instance(instances[i])

J = instance['jobs'][0]
T = instance['tamanho'][0]

with open('../97_9_opts.pkl', 'rb') as f:
    opts = pickle.load(f)

x_opt = opts[instances[i].name]['sol']

x_cand_dummy = np.ones_like(x_cand_opt, dtype=int)

# solve problem without coupling constraints
model = get_model(jobs, instance, coupling=False, new_ineq=False)

x_cand_feasible = get_feasible(model, x_cand_dummy, instance)
x_cand_feasible.sum()

../data/raw/97_9.jl


617

In [37]:
cut_w, cut_b = get_benders_cut(instance, x_cand_feasible)

print((cut_w * x_cand_opt).sum() + cut_b >= 0)
print((cut_w * x_cand_feasible).sum() + cut_b >= 0)

model.addConstr(
    sum(cut_w[j,t] * model.getVarByName(f"x({j},{t})")
        for j in range(J) for t in range(T))
    + cut_b >= 0
)
model.update()
new_x_cand_feasible = get_feasible(model, x_cand_feasible, instance)

print((cut_w * new_x_cand_feasible).sum() + cut_b >= 0)

new_x_cand_feasible.sum()

Set parameter InfUnbdInfo to value 1
True
False
True


603

In [38]:
cut_w, cut_b = get_benders_cut(instance, new_x_cand_feasible)

print((cut_w * x_cand_opt).sum() + cut_b >= 0)
print((cut_w * x_cand_feasible).sum() + cut_b >= 0)
print((cut_w * new_x_cand_feasible).sum() + cut_b >= 0)

model.addConstr(
    sum(cut_w[j,t] * model.getVarByName(f"x({j},{t})")
        for j in range(J) for t in range(T))
    + cut_b >= 0
)
model.update()
new_new_x_cand_feasible = get_feasible(model, new_x_cand_feasible, instance)

print((cut_w * new_new_x_cand_feasible).sum() + cut_b >= 0)

new_new_x_cand_feasible.sum()

Set parameter InfUnbdInfo to value 1
True
False
False
True


594

In [39]:
cut_w, cut_b = get_benders_cut(instance, new_new_x_cand_feasible)

print((cut_w * x_cand_opt).sum() + cut_b >= 0)
print((cut_w * x_cand_feasible).sum() + cut_b >= 0)
print((cut_w * new_x_cand_feasible).sum() + cut_b >= 0)
print((cut_w * new_new_x_cand_feasible).sum() + cut_b >= 0)

model.addConstr(
    sum(cut_w[j,t] * model.getVarByName(f"x({j},{t})")
        for j in range(J) for t in range(T))
    + cut_b >= 0
)
model.update()
new_new_new_x_cand_feasible = get_feasible(model, new_new_x_cand_feasible, instance)

print((cut_w * new_new_new_x_cand_feasible).sum() + cut_b >= 0)

new_new_new_x_cand_feasible.sum()

Set parameter InfUnbdInfo to value 1


TypeError: cannot unpack non-iterable NoneType object

In [42]:
(new_new_x_cand_feasible != x_cand_opt).sum()

587